In [1]:
import pandas as pd 
import numpy as np 
import sqlite3

## Data for Both Gender and All AGES

In [11]:
# download data based on the table in wiki page
# https://pardeewiki.du.edu//index.php?title=IHME
df = pd.read_csv('IHME-GBD_2019_DATA-0714f84e-1.csv')
df

,measure_id,measure_name,location_id,location_name,sex_id,sex_name,age_id,age_name,cause_id,cause_name,metric_id,metric_name,year,val,upper,lower
0,1,Deaths,18,Thailand,2,Female,22,All ages,718,Self-harm,3,Rate,1990,7.583004,8.945865,6.377517
1,1,Deaths,18,Thailand,3,Both,22,All ages,718,Self-harm,3,Rate,1990,12.385248,14.595850,10.375401
2,1,Deaths,18,Thailand,2,Female,22,All ages,724,Interpersonal violence,3,Rate,1990,7.849598,10.366865,5.846272
3,1,Deaths,18,Thailand,3,Both,22,All ages,724,Interpersonal violence,3,Rate,1990,16.075359,19.957210,11.981786
4,1,Deaths,129,Honduras,2,Female,22,All ages,945,Conflict and terrorism,3,Rate,1990,0.042123,0.046347,0.038247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48955,1,Deaths,205,Côte d'Ivoire,3,Both,22,All ages,945,Conflict and terrorism,3,Rate,2019,0.038209,0.042041,0.034694
48956,1,Deaths,156,United Arab Emirates,2,Female,22,All ages,945,Conflict and terrorism,3,Rate,2019,0.000000,0.000000,0.000000
48957,1,Deaths,156,United Arab Emirates,3,Both,22,All ages,945,Conflict and terrorism,3,Rate,2019,0.000000,0.000000,0.000000
48958,1,Deaths,27,Samoa,2,Female,22,All ages,945,Conflict and terrorism,3,Rate,2019,0.000000,0.000000,0.000000


### ViolencePoliticalConflictTerror: Death rate from conflict and terrorism IHME


In [49]:
#pd.DataFrame(list(dt.location_name.unique())).to_excel('Country.xlsx', header = ['IHME_Country'], index = False)
dt = df[df['sex_name'] == 'Both']
print(dt.cause_name.unique())
dt = dt[dt['cause_name'] == 'Conflict and terrorism']
dt = dt[['location_name','val','year']]
dt = dt.round({'val': 4})
first_last = dt.sort_values(['location_name', 'year']).groupby(dt['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
dt = pd.pivot_table(dt, index = 'location_name', columns= ['year'], values='val')
dt = dt.reset_index()
country = pd.read_excel('Country.xlsx')
dt = country.merge(dt, left_on='IHME_Country', right_on='location_name', how ='left')
dt = dt.merge(first_last,on='location_name',how='left')
dt = dt.drop(columns= ['IHME_Country','location_name'])
print(dt.columns)
dt


['Self-harm' 'Interpersonal violence' 'Conflict and terrorism'
 'Executions and police conflict']
Index([   'Country',  'FIPS_CODE',         1990,         1991,         1992,
               1993,         1994,         1995,         1996,         1997,
               1998,         1999,         2000,         2001,         2002,
               2003,         2004,         2005,         2006,         2007,
               2008,         2009,         2010,         2011,         2012,
               2013,         2014,         2015,         2016,         2017,
               2018,         2019,   'Earliest', 'MostRecent'],
      dtype='object')


,Country,FIPS_CODE,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,Earliest,MostRecent
0,Afghanistan,AFG,13.0486,26.7734,29.8704,25.6240,53.5389,31.5892,17.8528,35.6216,...,36.8303,37.0633,54.9809,70.4720,80.8051,53.8973,73.0467,63.4703,13.0486,63.4703
1,Albania,ALB,0.0000,0.0000,0.0299,0.0597,0.5372,0.0599,0.2404,59.2340,...,0.0000,0.0000,0.0358,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,Algeria,DZA,0.0158,0.2929,1.9175,3.0335,8.7218,12.0496,5.5884,16.6095,...,0.7733,0.5671,0.2944,0.2846,0.2300,0.2634,0.1844,0.0311,0.0158,0.0311
3,Angola,AGO,26.7098,5.5705,21.9207,108.2194,34.6920,2.1701,0.7287,22.0641,...,0.0255,0.0163,0.0000,0.0038,0.1320,0.0957,0.1053,0.0531,26.7098,0.0531
4,Argentina,ARG,0.4891,0.3400,0.4217,0.3646,0.3810,0.1143,0.1500,0.0084,...,0.0000,0.0000,0.0000,0.0000,0.0023,0.0000,0.0000,0.0000,0.4891,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,0.3399,0.3991,1.1640,0.3908,1.1325,0.1706,0.2135,0.0273,...,0.0000,0.0000,0.0034,0.0033,0.0478,0.0996,0.0000,0.0000,0.3399,0.0000
184,Viet Nam,VNM,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0013,...,0.0048,0.0110,0.0047,0.0062,0.0042,0.0083,0.0031,0.0000,0.0000,0.0000
185,Yemen,YEM,0.0000,0.0070,0.1634,0.0000,9.6001,0.0248,0.0180,0.0642,...,12.6945,4.3998,10.6894,26.8477,51.1714,55.6481,99.7647,42.5419,0.0000,42.5419
186,Zambia,ZMB,0.6547,0.0000,0.0000,0.0118,0.0000,0.0000,0.0000,0.0108,...,0.0754,0.0549,0.0384,0.0434,0.0195,0.0058,0.0283,0.0068,0.6547,0.0068


### ViolencePoliticalPoliceExecution: Death rate from police executions and conflict IHME

In [45]:
dt = df[df['sex_name'] == 'Both']
print(dt.cause_name.unique())
dt = dt[dt['cause_name'] == 'Executions and police conflict']
dt = dt[['location_name','val','year']]
dt = dt.round({'val': 4})
first_last = dt.sort_values(['location_name', 'year']).groupby(dt['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
dt = pd.pivot_table(dt, index = 'location_name', columns= ['year'], values='val')
dt = dt.reset_index()
country = pd.read_excel('Country.xlsx')
dt = country.merge(dt, left_on='IHME_Country', right_on='location_name', how ='left')
dt = dt.merge(first_last,on='location_name',how='left')
dt = dt.drop(columns= ['IHME_Country','location_name'])
print(dt.columns)
dt

['Self-harm' 'Interpersonal violence' 'Conflict and terrorism'
 'Executions and police conflict']
Index([   'Country',  'FIPS_CODE',         1990,         1991,         1992,
               1993,         1994,         1995,         1996,         1997,
               1998,         1999,         2000,         2001,         2002,
               2003,         2004,         2005,         2006,         2007,
               2008,         2009,         2010,         2011,         2012,
               2013,         2014,         2015,         2016,         2017,
               2018,         2019,   'Earliest', 'MostRecent'],
      dtype='object')


,Country,FIPS_CODE,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,Earliest,MostRecent
0,Afghanistan,AFG,0.0806,0.0808,0.0819,0.0845,0.0870,0.0894,0.0921,0.0951,...,0.0962,0.0561,0.0489,0.0514,0.0652,1.2348,0.4094,0.2780,0.0806,0.2780
1,Albania,ALB,0.0271,0.0363,0.0469,0.0678,0.0816,0.0852,0.0826,0.0793,...,0.0115,0.0115,0.0116,0.0118,0.0119,0.0119,0.0484,0.0118,0.0271,0.0118
2,Algeria,DZA,0.0830,0.0839,0.0874,0.0912,0.0964,0.1006,0.1049,1.5011,...,0.0796,0.1647,0.1116,0.0657,0.0697,0.0874,0.0807,0.0653,0.0830,0.0653
3,Angola,AGO,0.0077,0.0077,0.0078,0.0080,0.0081,0.0080,0.0075,0.0074,...,0.0187,0.0299,0.0103,0.0104,0.0357,0.0492,0.0924,0.0171,0.0077,0.0171
4,Argentina,ARG,0.0764,0.0827,0.0888,0.0978,0.1069,0.1185,0.1310,0.1535,...,0.0109,0.0109,0.0108,0.0109,0.0108,0.0108,0.0108,0.0108,0.0764,0.0108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,0.0674,0.1011,0.1548,0.2255,0.3146,0.4377,0.5955,0.7117,...,0.4477,0.4825,0.5075,0.5469,0.5715,1.1566,0.6084,0.6247,0.0674,0.6247
184,Viet Nam,VNM,0.0238,0.0245,0.0253,0.0260,0.0268,0.0274,0.0280,0.0285,...,0.0340,0.0411,0.0343,0.0349,0.0353,0.0365,0.1242,0.0354,0.0238,0.0354
185,Yemen,YEM,0.0496,0.0504,0.0515,0.0529,0.0550,0.0569,0.0590,0.0613,...,0.1471,0.0898,0.0415,0.3938,0.0513,0.0509,0.0965,0.0697,0.0496,0.0697
186,Zambia,ZMB,0.0093,0.0094,0.0096,0.0096,0.0097,0.0098,0.0098,0.0097,...,0.0192,0.0456,0.0192,0.0191,0.0369,0.0192,0.0414,0.0297,0.0093,0.0297


### ViolenceSelfHarm: Death rate from self harm IHME

In [12]:
dt = df[df['sex_name'] == 'Both']
print(dt.cause_name.unique())
dt = dt[dt['cause_name'] == 'Self-harm']
dt = dt[['location_name','val','year']]
dt = dt.round({'val': 4})
first_last = dt.sort_values(['location_name', 'year']).groupby(dt['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
dt = pd.pivot_table(dt, index = 'location_name', columns= ['year'], values='val')
dt = dt.reset_index()
country = pd.read_excel('Country.xlsx')
dt = country.merge(dt, left_on='IHME_Country', right_on='location_name', how ='left')
dt = dt.merge(first_last,on='location_name',how='left')
dt = dt.drop(columns= ['IHME_Country','location_name'])
print(dt.columns)
dt

['Self-harm' 'Interpersonal violence' 'Conflict and terrorism'
 'Executions and police conflict']
Index([   'Country',  'FIPS_CODE',         1990,         1991,         1992,
               1993,         1994,         1995,         1996,         1997,
               1998,         1999,         2000,         2001,         2002,
               2003,         2004,         2005,         2006,         2007,
               2008,         2009,         2010,         2011,         2012,
               2013,         2014,         2015,         2016,         2017,
               2018,         2019,   'Earliest', 'MostRecent'],
      dtype='object')


,Country,FIPS_CODE,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,Earliest,MostRecent
0,Afghanistan,AFG,6.0961,5.9626,5.8761,5.9006,5.9317,5.8978,5.8683,5.8508,...,4.5856,4.4890,4.4095,4.3686,4.3346,4.3096,4.2684,4.2145,6.0961,4.2145
1,Albania,ALB,3.7733,4.0887,3.9597,3.9064,3.7339,3.9789,4.2472,4.6067,...,5.5565,5.5690,5.6566,5.7464,5.7156,5.6708,5.6292,5.5913,3.7733,5.5913
2,Algeria,DZA,5.5588,5.3493,5.2860,5.2460,5.2943,5.2770,5.2474,5.2351,...,3.9193,3.8669,3.8343,3.7690,3.6437,3.5730,3.6117,3.6215,5.5588,3.6215
3,Angola,AGO,9.4256,9.2806,9.2342,9.3318,9.3814,9.1178,8.4860,8.2282,...,6.8050,6.7055,6.4816,6.4217,6.3223,6.3876,6.4107,6.3957,9.4256,6.3957
4,Argentina,ARG,8.3262,8.5756,9.0389,9.3185,9.6503,9.9909,10.4890,11.1114,...,11.3692,11.3841,11.1775,10.9818,11.2153,11.2247,11.1950,11.1469,8.3262,11.1469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,"Venezuela, Bolivarian Republic",VEN,5.6001,5.9447,6.3365,6.8457,7.4893,7.8973,7.9689,7.8228,...,7.8352,7.9267,8.0494,8.1574,8.0445,8.1165,8.2566,8.2292,5.6001,8.2292
185,Viet Nam,VNM,8.1060,8.0365,7.9462,7.8672,7.7856,7.7356,7.6542,7.5799,...,7.9738,8.0841,8.2067,8.2701,8.3463,8.4098,8.4297,8.4194,8.1060,8.4194
186,Yemen,YEM,4.3432,4.2544,4.2089,4.1899,4.2069,4.1890,4.1874,4.2095,...,4.4442,4.4943,4.4117,4.5127,4.6088,4.7308,4.9155,4.9524,4.3432,4.9524
187,Zambia,ZMB,7.3472,7.6679,7.9445,8.1683,8.4130,8.6520,8.7638,8.7940,...,7.7412,7.7198,7.7418,7.7552,7.7594,7.8137,7.7659,7.6637,7.3472,7.6637


### SocietalViolenceDeathsTotal: Societal violence deaths total (Interpersonal+Conflict+Policeexecutions)

In [58]:
dt = df[df['sex_name'] == 'Both']
print(dt.cause_name.unique())
dt = dt[dt['cause_name'] != 'Self-harm']
dt = dt[['location_name','val','year']]
dt = dt.groupby(by=["location_name",'year'], dropna=False).sum()
dt = dt.reset_index()
dt = dt.round({'val': 4})
first_last = dt.sort_values(['location_name', 'year']).groupby(dt['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
dt = pd.pivot_table(dt, index = 'location_name', columns= ['year'], values='val')
dt = dt.reset_index()
country = pd.read_excel('Country.xlsx')
dt = country.merge(dt, left_on='IHME_Country', right_on='location_name', how ='left')
dt = dt.merge(first_last,on='location_name',how='left')
dt = dt.drop(columns= ['IHME_Country','location_name'])
print(dt.columns)
dt

['Self-harm' 'Interpersonal violence' 'Conflict and terrorism'
 'Executions and police conflict']
Index([   'Country',  'FIPS_CODE',         1990,         1991,         1992,
               1993,         1994,         1995,         1996,         1997,
               1998,         1999,         2000,         2001,         2002,
               2003,         2004,         2005,         2006,         2007,
               2008,         2009,         2010,         2011,         2012,
               2013,         2014,         2015,         2016,         2017,
               2018,         2019,   'Earliest', 'MostRecent'],
      dtype='object')


,Country,FIPS_CODE,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,Earliest,MostRecent
0,Afghanistan,AFG,26.5941,42.7552,45.7590,41.9051,70.6511,48.6522,36.2131,51.7613,...,50.7004,50.8386,68.5253,85.5283,95.3672,69.5849,88.2934,76.8510,26.5941,76.8510
1,Albania,ALB,5.3767,6.2595,6.9979,7.8381,9.1411,9.9362,11.4105,72.3688,...,2.2729,2.2541,2.2920,2.2696,2.2388,2.1955,2.1995,2.1060,5.3767,2.1060
2,Algeria,DZA,2.0417,2.2426,3.8734,5.0072,10.8056,14.2613,7.6213,20.9252,...,2.2432,2.1459,1.7207,1.6179,1.5156,1.5187,1.4007,1.1943,2.0417,1.1943
3,Angola,AGO,31.1564,9.9362,26.2723,112.6407,39.1707,6.5191,4.6943,25.8883,...,3.7128,3.6558,3.5222,3.4525,3.4714,3.4495,3.4511,3.3025,31.1564,3.3025
4,Argentina,ARG,6.6759,6.4094,6.7191,6.6548,6.9143,6.4418,6.7202,6.2668,...,6.5316,6.7071,6.8929,6.6960,6.6551,6.5495,6.4720,6.3977,6.6759,6.3977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,15.4880,17.3394,19.3974,21.0786,24.4947,22.9192,22.1111,20.3806,...,47.6688,47.2767,45.3044,45.0707,43.2931,42.0212,40.9389,39.6151,15.4880,39.6151
184,Viet Nam,VNM,2.4235,2.4283,2.4202,2.4188,2.4157,2.3886,2.3535,2.3318,...,1.7644,1.7709,1.4743,1.4667,1.4467,1.4209,1.4737,1.3499,2.4235,1.3499
185,Yemen,YEM,1.9731,1.9576,2.1007,1.9531,11.5896,2.0113,2.0135,2.0826,...,15.3208,6.9428,14.7682,31.0081,56.9245,59.4194,103.4871,45.2132,1.9731,45.2132
186,Zambia,ZMB,6.3757,5.9672,6.2965,6.5418,6.8349,7.1476,7.3550,7.5922,...,8.9054,8.8811,8.8485,8.9196,9.0109,8.9402,8.9786,8.8949,6.3757,8.8949


## Interperson: Child/WomanAdult

### ViolenceInterpersonChild

In [42]:
# download data based on the table in wiki page
# https://pardeewiki.du.edu//index.php?title=IHME
dt = pd.read_excel('ViolenceInterpersonChild.xlsx')
country = pd.read_excel('Country.xlsx', sheet_name='Interperson')
dt = dt[['location_name','val','year']]
dt = dt.groupby(by=["location_name",'year'], dropna=False).sum()
dt = dt.reset_index()
dt = dt.round({'val': 4})
dt1 = dt
first_last = dt1.sort_values(['location_name','year']).groupby(dt1['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
first_last = first_last.merge(country, left_on= 'location_name', right_on='IHME_Country', how = 'left')
first_last = first_last.drop(columns= ['IHME_Country','location_name','FIPS_CODE'])
dt = pd.pivot_table(dt, index = 'location_name', columns= ['year'], values='val')
dt = dt.reset_index() # release index to merge with country
dt = country.merge(dt, left_on='IHME_Country', right_on='location_name', how ='left')
dt = dt.drop(columns= ['IHME_Country','location_name'])
dt2 = dt[dt['Country'] == "Cote D'Ivoire"]
dt = dt[dt['Country'] != "Cote D'Ivoire"]
dt2 = dt2.max().to_frame().T
dt = dt.append(dt2).sort_values('Country')
dt = dt.merge(first_last, on=['Country'],how='left')
dt

C:\Users\panda\AppData\Local\Temp\ipykernel_19040\1426077035.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt = dt.append(dt2).sort_values('Country')


,Country,FIPS_CODE,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,Earliest,MostRecent
0,Afghanistan,AFG,3.401,3.8218,3.8048,3.8699,3.9416,3.8203,3.9821,3.6036,...,2.7225,2.8621,2.8434,3.063,2.9883,2.9347,4.0275,2.6223,3.4010,2.6223
1,Albania,ALB,1.04,1.2011,1.2798,1.3836,1.4685,1.6033,1.7381,1.9443,...,0.5585,0.5604,0.5652,0.5695,0.5603,0.5468,0.5303,0.5127,1.0400,0.5127
2,Algeria,DZA,0.483,0.4564,0.4462,0.4385,0.4408,0.4518,0.4149,0.5573,...,0.2701,0.2728,0.2567,0.2452,0.233,0.2204,0.2151,0.2075,0.4830,0.2075
3,Angola,AGO,1.7491,1.7337,1.735,1.751,1.7593,1.7353,1.672,1.6558,...,1.4579,1.3967,1.3223,1.2831,1.2007,1.1597,1.1193,1.115,1.7491,1.1150
4,Argentina,ARG,1.394,1.324,1.4031,1.3516,1.3799,1.2905,1.284,0.9926,...,1.1322,1.1062,1.1349,1.064,1.0549,0.96,0.9201,0.8871,1.3940,0.8871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,1.7407,1.7777,1.8539,2.0956,2.4207,2.893,2.8289,2.6006,...,4.1629,4.1978,3.9298,3.8081,3.8046,3.7635,3.7198,3.6052,1.7407,3.6052
184,Viet Nam,VNM,0.3651,0.3643,0.3615,0.3586,0.3544,0.3487,0.3431,0.3393,...,0.3003,0.289,0.2203,0.2195,0.2138,0.2063,0.1984,0.1902,0.3651,0.1902
185,Yemen,YEM,0.7069,0.6984,0.6935,0.6912,0.6866,0.6816,0.6771,0.6746,...,0.6607,0.6619,0.9232,0.8793,1.1911,0.8626,1.4061,0.6607,0.7069,0.6607
186,Zambia,ZMB,2.9487,2.9729,3.031,3.0242,3.0301,3.0181,2.9364,2.852,...,2.0473,2.0013,1.9409,1.8702,1.7617,1.7543,1.6727,1.6366,2.9487,1.6366


### ViolenceInterpersonWomenAdult

In [43]:
# download data based on the table in wiki page
# https://pardeewiki.du.edu//index.php?title=IHME
dt = pd.read_excel('ViolenceInterpersonWomenAdult.xlsx')
country = pd.read_excel('Country.xlsx', sheet_name='Interperson')
dt = dt[['location_name','val','year']]
dt = dt.groupby(by=["location_name",'year'], dropna=False).sum()
dt = dt.reset_index()
dt = dt.round({'val': 4})
dt1 = dt
first_last = dt1.sort_values(['location_name','year']).groupby(dt1['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
first_last = first_last.merge(country, left_on= 'location_name', right_on='IHME_Country', how = 'left')
first_last = first_last.drop(columns= ['IHME_Country','location_name','FIPS_CODE'])
dt = pd.pivot_table(dt, index = 'location_name', columns= ['year'], values='val')
dt = dt.reset_index() # release index to merge with country
dt = country.merge(dt, left_on='IHME_Country', right_on='location_name', how ='left')
dt = dt.drop(columns= ['IHME_Country','location_name'])
dt2 = dt[dt['Country'] == "Cote D'Ivoire"]
dt = dt[dt['Country'] != "Cote D'Ivoire"]
dt2 = dt2.max().to_frame().T
dt = dt.append(dt2).sort_values('Country')
dt = dt.merge(first_last, on=['Country'],how='left')
dt

C:\Users\panda\AppData\Local\Temp\ipykernel_19040\392065484.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt = dt.append(dt2).sort_values('Country')


,Country,FIPS_CODE,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,Earliest,MostRecent
0,Afghanistan,AFG,6.0807,6.7293,6.1525,5.9556,6.2535,6.2429,6.8361,5.974,...,4.7646,4.6954,4.6336,5.223,4.97,4.9622,6.926,4.4073,6.0807,4.4073
1,Albania,ALB,2.4204,2.6317,2.702,2.7845,2.9142,3.3326,3.75,4.3659,...,1.147,1.1357,1.1191,1.1079,1.0973,1.0847,1.075,1.0575,2.4204,1.0575
2,Algeria,DZA,1.2608,1.2224,1.2045,1.1927,1.2013,1.2357,1.151,1.4598,...,0.7708,0.7801,0.7431,0.7225,0.7014,0.6821,0.6676,0.65,1.2608,0.6500
3,Angola,AGO,2.3171,2.2826,2.2724,2.3243,2.3692,2.3079,2.1332,2.0713,...,2.0207,1.9886,1.9245,1.8884,1.848,1.846,1.8263,1.7964,2.3171,1.7964
4,Argentina,ARG,2.4407,2.4774,2.7071,2.7493,2.8343,2.6348,2.6558,2.5717,...,2.3471,2.3873,2.391,2.3857,2.5023,2.3872,2.34,2.308,2.4407,2.3080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,3.5104,4.2843,4.1105,4.409,5.2732,6.9008,6.5352,6.3638,...,8.3011,8.2244,11.0564,11.5428,10.4036,10.1458,10.3819,10.0365,3.5104,10.0365
184,Viet Nam,VNM,1.0946,1.085,1.0732,1.0616,1.0471,1.029,1.0046,0.9811,...,0.6568,0.649,0.3884,0.3848,0.3812,0.3761,0.3713,0.3668,1.0946,0.3668
185,Yemen,YEM,1.4834,1.4662,1.4511,1.4479,1.4605,1.4386,1.4173,1.4012,...,1.5251,1.5219,2.1612,2.0625,2.8505,2.0516,2.901,1.6011,1.4834,1.6011
186,Zambia,ZMB,4.4998,4.6665,4.8182,4.9226,5.0868,5.2366,5.3648,5.5137,...,6.0857,5.9654,5.863,5.7881,5.7648,5.6828,5.595,5.4977,4.4998,5.4977


# HIV & Malaria

## Death Number

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_HIV_Malaria_Death_Num.csv')
dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='HIV/AIDS']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

dt0['val'] = round(dt0['val']/1000000,5)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
dt1

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_HIV_Malaria_Death_Num.csv')
dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

dt0 = dt[dt['cause_name']=='Malaria']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

dt0['val'] = round(dt0['val']/1000000,5)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
dt1


## Prevalence Number

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_HIV_Malaria_Pre_Num.csv')
dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='HIV/AIDS']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

# dt0['val'] = round(dt0['val']/1000000,5)
dt0['val'] = round(dt0['val'], 0)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
dt1

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_HIV_Malaria_Pre_Num.csv')
dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='Malaria']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

# dt0['val'] = round(dt0['val']/1000000,5)
dt0['val'] = round(dt0['val'], 0)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
dt1

## Incidence Number

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_HIV_Malaria_Inc_Num.csv')
dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='HIV/AIDS']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

# dt0['val'] = round(dt0['val']/1000000,5)
dt0['val'] = round(dt0['val'], 0)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
# print(dt1.head())
# dt1.sort_values(by= [1990],ascending=False)
dt1

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_HIV_Malaria_Inc_Num.csv')
dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='Malaria']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

# dt0['val'] = round(dt0['val']/1000000,5)
dt0['val'] = round(dt0['val'], 0)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
#print(dt1.head())
#dt1.sort_values(by= [1990],ascending=False)
dt1

# Prevelance Percent

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_Prevalence_Percent.csv')
#dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='Amphetamine use disorders']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

# dt0['val'] = round(dt0['val']/1000000,5)
dt0['val'] = round(dt0['val']*100, 5)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
# print(dt1.head())
# dt1.sort_values(by= [1990],ascending=False)
dt1

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_Prevalence_Percent.csv')
#dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='Cocaine use disorders']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

# dt0['val'] = round(dt0['val']/1000000,5)
dt0['val'] = round(dt0['val']*100, 5)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
# print(dt1.head())
# dt1.sort_values(by= [1990],ascending=False)
dt1

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_Prevalence_Percent.csv')
#dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='HIV/AIDS']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

# dt0['val'] = round(dt0['val']/1000000,5)
dt0['val'] = round(dt0['val'], 3)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
# print(dt1.head())
# dt1.sort_values(by= [1990],ascending=False)
dt1

In [ ]:
dt = pd.read_csv('IHME-GBD_2019_Prevalence_Percent.csv')
#dt = dt[dt['metric_name'] == 'Number']
dt = dt[['location_name','cause_name','year','val']]

print(dt.cause_name.unique())

dt0 = dt[dt['cause_name']=='Opioid use disorders']
dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

# dt0['val'] = round(dt0['val']/1000000,5)
dt0['val'] = round(dt0['val'], 4)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year',values = 'val').reset_index()
first_last = dt0.sort_values(['location_name','year']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
# print(dt1.head())
# dt1.sort_values(by= [1990],ascending=False)
dt1

# Under 5 Mortality

In [ ]:
level2country = pd.read_excel('IHME_Location_ID_Name.xlsx')
dt = pd.read_csv('IHME_GBD_2019_MORTALITY_1950_2019_5Q0_NOSHOCK_Y2020M07D31.CSV')
# print(dt.columns)
c = list(level2country.location_id)
print(dt.shape)
dt = dt[dt['location_id'].isin(c)]
print(dt.shape)
dt = dt[dt['sex_name'] == 'both']
# dt = dt[dt['location_id'] != 533]
dt0 = dt[['location_name','year_id','val']]
# print(dt.columns)

dt0['val'] = dt0['val'].apply(pd.to_numeric, errors='coerce')
country = pd.read_excel('Country.xlsx')

dt0['val'] = round(dt0['val']*1000, 4)

print(dt0.columns)

dt1 = pd.pivot_table(dt0,index = ["location_name"],columns='year_id',values = 'val').apply(pd.to_numeric, errors='coerce').reset_index()
first_last = dt0.sort_values(['location_name','year_id']).groupby(dt0['location_name'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})

dt1 = dt1.merge(first_last, on = 'location_name', how = 'left')
dt1 = country.merge(dt1, left_on='IHME_Country', right_on='location_name', how ='left')
dt1 = dt1.drop(columns=['IHME_Country','location_name'])
# print(dt1.head())
# dt1.sort_values(by= [1950],ascending=False)
dt1


# Infant Mortality

In [ ]:
ys = []
ye = []
for i in range(1950,2020,10):
    ys.append(i)
    ye.append(i+9)

level2country = pd.read_excel('IHME_Location_ID_Name.xlsx')
c = list(level2country.location_id)
for j in range(len(ys)):
    df = pd.read_csv(f'Infant/IHME_GBD_2019_U5M_{ys[j]}_{ye[j]}_POD_Y2021M09D01.CSV')
    print('Infant Mortality', ys[j],ye[j])
    df = df[df['location_id'].isin(c)]
    df = df[df['sex']=='Both']
    # df = df[df['location_id'] != 533]
    df = df[['location_name','age_group_name','year_id','val']]
    # print(df.age_group_name.unique())
    dt = df[df['age_group_name'].isin(['1-5 months','6-11 months','Neonatal'])]
    # dt = dt.groupby(['location_name','year_id']).sum().reset_index()
    dt['val'] = dt['val'].apply(pd.to_numeric, errors='coerce')
    dt.val = round(dt.val*1000,4) 
    country = pd.read_excel('Country.xlsx', sheet_name='Country2')
    dt = country.merge(dt, left_on = 'IHME_Country', right_on = 'location_name', how = 'left' )
    dt = dt.drop(columns=['IHME_Country','location_name','FIPS_CODE'])
    if j == 0:
        final = dt
    else:
        final = pd.concat([final,dt])
# print(final.year_id.unique())

final.year_id = final.year_id.astype("Int64")
final.year_id = final.year_id.map(str)

final = final.groupby(['Country','year_id']).sum().reset_index()

# final = final.reset_index()
first_last = final.sort_values(['Country','year_id']).groupby(final['Country'])['val'] \
        .agg(['first', 'last']).reset_index()
first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})


final_wide = pd.pivot_table(final,index = ["Country"],columns='year_id',values = 'val').reset_index()
final_wide = final_wide.merge(first_last, on = 'Country', how = 'left')
final_wide = country.merge(final_wide, on = 'Country', how = 'left')
final_wide = final_wide.drop(columns=['IHME_Country'])

dt1 = final_wide
dt1#.to_csv('InfMortRateIHME.csv', index = False)


# Life Expect

In [ ]:
gender = ['male', 'female', 'both']
level2country = pd.read_excel('IHME_Location_ID_Name.xlsx')
c = list(level2country.location_id)
for g in gender:
    # download data from IHME based on the wiki page
    # https://pardeewiki.du.edu//index.php?title=IHME
    df = pd.read_csv(f'IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_28_NOSHOCK_Y2020M07D31.CSV')
    df = df[df['metric_name']=='Years']
    df = df[df['location_id'].isin(c)]
    # df = df[df['location_id'] != 533]
    df = df[df['sex_name']==f'{g}']
    df = df[['location_name','age_group_name','year_id','val']]
    # print(df[df['location_name']=="Côte d'Ivoire"])
    # if g == 'male':
    #     dt = df
    # else:
    #     dt = pd.concat([dt,df])
    dd = df.groupby(['location_name','year_id']).mean().reset_index()
    dd.year_id = dd.year_id.astype("Int64")
    dd.year_id = dd.year_id.map(str)
    dd.val = round(dd.val,3)
    first_last = dd.sort_values(['location_name','year_id']).groupby(dd['location_name'])['val'] \
            .agg(['first', 'last']).reset_index()
    first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
    final = pd.pivot_table(dd,index = ["location_name"],columns='year_id',values = 'val').reset_index()
    country = pd.read_excel('Country.xlsx')
    final = country.merge(final, left_on = 'IHME_Country', right_on = 'location_name', how = 'left' )
    final = final.merge(first_last, on = 'location_name', how = 'left')
    final = final.drop(columns=['IHME_Country','location_name'])
    print(f'LifeExpectIHME{g.capitalize()}HistOnly')
    # final.to_csv(f'LifeExpectIHME{g.capitalize()}HistOnly.csv', index = False)

# HealthDetailedDeathsCtry in IFs.db

In [ ]:
country = pd.read_excel('Country.xlsx')
age = pd.read_excel('AgeID_AgeName.xlsx')
cause = pd.read_excel('CauseID_Cause.xlsx')
df = pd.read_csv('IHME-GBD_2019_DATA-b5768275-1.csv')
df = df[df['sex_name'] !='Both']
# df = df[df['year']== 2019]
print(df.year.unique())
df = df[['location_name','sex_id','age_id','cause_name','val']]
print(df.sex_id.unique())
df.val = df.val/100
dt = df.merge(age, on = 'age_id', how = 'left')
dt = dt.merge(cause,on = 'cause_name', how = 'left')
dt = dt.rename(columns={'sex_id': 'sex',
                        'Cause_ID': 'CauseID'})
dt = dt.pivot_table(index=['location_name','sex','CauseID','Cause'],
               columns=['Age'],
               values='val',
               aggfunc='sum')
col = []
for i in range(1,22):
    col.append('p'+str(i))

dt = dt[col]

dt = dt.reset_index()
# dt[(dt['location_name'] == 'United States of America') & (dt['CauseID']==3)]

dt = country.merge(dt, left_on = 'IHME_Country', right_on = 'location_name', how = 'left')
dt = dt.drop(columns=['IHME_Country','location_name','FIPS_CODE'])

dt.sex = dt.sex.astype("Int64")
dt